# Create a Foot Traffic Data Set by street 

Based off the XPlore logic, this time load in more more foot traffic data based on the full history downloaded and saved to /data_files_raw/foot_traffic_melb/ folder

This time, rather than aggregate all the street numbers to get a total melbourne number, cherry pick some of the streets. Convert it to a long format with the street name

In [ ]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import re

from utilities import data_basic_utility as databasic
from utilities import regex_utility as reutil

# File Variables

Set the start and end date of the time frame of data to load in and combine into one unified file

This is the list of all Streets that exist in all files from Jan 2013 to July 2022
['Date', 'Hour', 'Bourke Street Mall (North)', 'Bourke Street Mall (South)', 'Melbourne Central', 'Town Hall (West)', 'Princes Bridge', 'Birrarung Marr', 'Webb Bridge', 'Southern Cross Station', 'Victoria Point', 'Waterfront City', 'New Quay', 'Flagstaff Station', 'Sandridge Bridge', 'State Library', 'Collins Place (South)', 'Collins Place (North)', 'Chinatown-Swanston St (North)', 'Flinders St-Elizabeth St (East)', 'Spencer St-Collins St (South)', 'Spencer St-Collins St (North)', 'QV Market-Peel St']

In [ ]:
filesEndDate = datetime(2022, 7, 1)
#filesStartDate = datetime(2022, 4, 1)
filesStartDate = datetime(2013, 1, 1)

footTrafficFolder = "./data_files_raw/foot_traffic_melb/"
tempFolder = "./tmp/"

#Create a list of streets to use. Try to get a good cross section of the city, but not too many locations
streetsToUse = [ 
    "Bourke Street Mall (North)",
    "Melbourne Central",
    "Southern Cross Station",
    "Chinatown-Swanston St (North)",
    "Spencer St-Collins St (North)",
    "QV Market-Peel St",
    "Collins Place (North)"
]

Load all the weather data files like rain and temp

In [ ]:
rain = pd.read_csv("IDCJAC0009_086338_1800_Data.csv")
max_temp = pd.read_csv("IDCJAC0010_086338_1800_Data.csv")
min_temp = pd.read_csv("IDCJAC0011_086338_1800_Data.csv")
solar_exp = pd.read_csv("IDCJAC0016_086338_1800_Data.csv")

In [ ]:
# Helper functions for data files
def make_date_col(df):
    df["date"] = df['Day'].astype(str) + "/" + df['Month'].astype(str) + "/" + df['Year'].astype(str)
    df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
    return df['date']

    
def filter_weathers(df, startDate='2022-07-01', endDate='2022-07-31'):
    df = df.loc[(df['date'] >= startDate) & (df['date'] <= endDate)]
    return df    

## Rain clean

In [ ]:
print(rain.info())
rain.head(10)

In [ ]:
rain = rain.drop(labels=['Product code', 'Bureau of Meteorology station number'], axis=1)

In [ ]:
rain["date"] = make_date_col(rain)

In [ ]:
rain.head(10)

In [ ]:
rain.rename(columns={'Rainfall amount (millimetres)':"total_rain"},inplace=True)

In [ ]:
rain.head()

In [ ]:
rain.sort_index(ascending=False).head()

## Foot traffic clean

In [ ]:
# Have a look at one of the files
foot_traffic = pd.read_csv("July_2022.csv")

print(foot_traffic.info())
foot_traffic.head(10)

Example of the working to convert a file so that it has counted up all the numbers for a day, but only for the streets to use, and then also pivoted into a longer format

In [ ]:
foot_traffic=foot_traffic.replace(to_replace=["na","undefined"],value=0)
columnsToUse = [ "Date", "Hour" ]
columnsToUse = columnsToUse + streetsToUse
#print(columnsToUse)

foot_traffic = foot_traffic[columnsToUse]
foot_traffic.head()

In [ ]:
dfFtStreet = pd.melt(foot_traffic, id_vars=[ "Date", "Hour" ], var_name="Street")
dfFtStreet = dfFtStreet.rename(columns = { "value":"people" })
dfFtStreet["people"] = dfFtStreet.apply(lambda x: int(x["people"]), axis=1)
print(dfFtStreet.info())
dfFtStreet.head(20)


In [ ]:
dfFtStreet = dfFtStreet.groupby([ "Date", "Street" ])["people"].sum().reset_index()

dfFtStreet.head(20)

In [ ]:
# This is basically Freddie's logic of loading in one month's file, cleaning it and converting it to a total people count
def loadAndCountFootTrafficFile(dataFile):
    foot_traffic = pd.read_csv(footTrafficFolder + dataFile)
    foot_traffic=foot_traffic.replace(to_replace=["na","undefined"],value=0)
    # foot_traffic["total_people"] = foot_traffic.iloc[:,2:].sum(axis=1)

    # get the first date to test the structure
    firstDate = foot_traffic["Date"][0]
    if reutil.re_is_match(reutil.regex_DateDdMmYyyy(), firstDate):
        foot_traffic["Date"] = pd.to_datetime(foot_traffic["Date"], format="%d/%m/%Y")
    elif reutil.re_is_match(reutil.regex_DateDdMmYy(), firstDate):
        foot_traffic["Date"] = pd.to_datetime(foot_traffic["Date"], format="%d/%m/%y")
    elif reutil.re_is_match(reutil.regex_DateDdMmmYy(), firstDate):
        foot_traffic["Date"] = pd.to_datetime(foot_traffic["Date"], format="%d-%b-%y")    

    # First, filter out the streets we don't want to use
    columnsToUse = [ "Date", "Hour" ]
    columnsToUse = columnsToUse + streetsToUse
    foot_traffic = foot_traffic[columnsToUse]

    # Then, unpivot the data by the date and hour, so we have records of Street name and total_people
    foot_traffic = pd.melt(foot_traffic, id_vars=[ "Date", "Hour" ], var_name="street")
    foot_traffic = foot_traffic.rename(columns = { "value":"total_people" })
    foot_traffic.loc[foot_traffic["total_people"].isna(), "total_people"] = 0
    foot_traffic["total_people"] = foot_traffic.apply(lambda x: int(x["total_people"]), axis=1)

    # Then aggregate all the hourly numbers so we have a count by street and day
    foot_traffic = foot_traffic.groupby([ "Date", "street" ])["total_people"].sum().reset_index()
    foot_traffic.rename(columns={'Date':'date'}, inplace=True)
    return foot_traffic



In [ ]:
# Test the function
dfFT_202207 = loadAndCountFootTrafficFile("July_2022.csv")

print(dfFT_202207.info())
dfFT_202207.head(10)

In [ ]:
# First, start with the end month, load the data into a dataframe
fileName = filesEndDate.strftime("%B") + "_" + filesEndDate.strftime("%Y") + ".csv"
print(fileName)

dfFootTraffic = loadAndCountFootTrafficFile(fileName)
print(dfFootTraffic.info())
dfFootTraffic.head()

In [ ]:
date1 = '1-Nov-13'
date2 = "23/5/21"

pattern = reutil.regex_DateDdMmmYy()
if re.match(pattern, date1):
    print("date1 regex_DateDdMmmYy MATCH")
else:
    print("date1 regex_DateDdMmmYy NO MATCH")


pattern = reutil.regex_DateDdMmmYy()
if re.match(pattern, date2):
    print("date2 regex_DateDdMmmYy MATCH")
else:
    print("date2 regex_DateDdMmmYy NO MATCH")

pattern = reutil.regex_DateDdMmYy()
if re.match(pattern, date2):
    print("date2 regex_DateDdMmmYy MATCH")
else:
    print("date2 regex_DateDdMmmYy NO MATCH")    

Find all streets that exist in all files

In [ ]:
filesEndDate = datetime(2022, 7, 1)
#filesStartDate = datetime(2022, 4, 1)
filesStartDate = datetime(2013, 1, 1)

In [ ]:
stepperDate = filesEndDate
stepperDate = stepperDate - relativedelta(months=1)
stepCount = 0

columnList = []

while stepperDate >= filesStartDate:
    fileName = stepperDate.strftime("%B") + "_" + stepperDate.strftime("%Y") + ".csv"

    foot_traffic = pd.read_csv(footTrafficFolder + fileName)

    if len(columnList) == 0:
        columnList = foot_traffic.columns
    else:
        # filter the list to include only if exists in other list
        # columnList = [x for x in columnList if x[0] in foot_traffic.columns]
        columnList = list(filter(lambda x: x in foot_traffic.columns, columnList))

    # step back
    stepperDate = stepperDate - relativedelta(months=1)

    # Sanity check, break in case of an infinite loop
    stepCount += 1
    if stepCount > 1000:
        break

print(columnList)

In [ ]:
# Now go back to the month before
stepperDate = filesEndDate
stepperDate = stepperDate - relativedelta(months=1)
stepCount = 0

# For each month, load the data, append it to the total dataframe then step back another month
# keep going until we get all the way to the start date
while stepperDate >= filesStartDate:
    fileName = stepperDate.strftime("%B") + "_" + stepperDate.strftime("%Y") + ".csv"
    # print(fileName)

    dfMonth = loadAndCountFootTrafficFile(fileName)
    dfFootTraffic = pd.concat([dfFootTraffic, dfMonth])

    # step back
    stepperDate = stepperDate - relativedelta(months=1)

    # Sanity check, break in case of an infinite loop
    stepCount += 1
    if stepCount > 1000:
        break


# Order by the date desc
dfFootTraffic = dfFootTraffic.sort_values(["date"], ascending=False)

print(dfFootTraffic.info())
dfFootTraffic.head()


In [ ]:
dfFootTraffic.sort_values(["date"], ascending=True).head()

dfFootTraffic.to_csv(tempFolder + "foottrafficstreet.csv")

# Join and plot 

In [ ]:
max_temp['date'] = make_date_col(max_temp)
min_temp['date'] = make_date_col(min_temp)
solar_exp['date'] = make_date_col(solar_exp)

In [ ]:
# Get out the range of dates that have been loaded from the foot traffic and filter the weather data accordingly
ftMinDate = dfFootTraffic["date"].min()
ftMaxDate = dfFootTraffic["date"].max()

print(ftMinDate)
print(ftMaxDate)

max_temp=filter_weathers(max_temp, ftMinDate, ftMaxDate)
min_temp=filter_weathers(min_temp, ftMinDate, ftMaxDate)
solar_exp=filter_weathers(solar_exp, ftMinDate, ftMaxDate)

In [ ]:
max_temp.rename(columns={"Maximum temperature (Degree C)":"max_temp"},inplace=True)
min_temp.rename(columns={"Minimum temperature (Degree C)":"min_temp"},inplace=True)
solar_exp.rename(columns={"Daily global solar exposure (MJ/m*m)":"solar_exp"},inplace=True)

In [ ]:
print(max_temp.info())
max_temp.head()


# max_temp.to_csv(tempFolder + "maxtemp.csv")

Write out a datafile with the date and the total foot traffic numbers in Melbourne, for future use

Also, create a datafile with the foot traffic and all the weather columns by day for Melbourne, we can use that later as a source datafile for basic modelling

In [ ]:
# Foot Traffic by day Data
outputFootTrafficFileName = "FootTrafficMelbStreet_" + filesStartDate.strftime("%Y%m%d") + "_" + filesEndDate.strftime("%Y%m%d") + ".csv"
dfFootTraffic.to_csv("./data_files/" + outputFootTrafficFileName, index=False)

print(dfFootTraffic.info())
dfFootTraffic.head()

In [ ]:
rain.head()

The data we want is total rain, and also Quality, which is a Y/N and shows whether the rain measurement has passed full quality control. If N, then the measurement might be suspect

In [ ]:
dfRainToMerge = rain[["total_rain", "Quality", "date"]]
dfRainToMerge.rename(columns={"Quality":"rain_quality"},inplace=True)

dfFootTrafficWeather = pd.merge(dfFootTraffic, dfRainToMerge, how="inner", on="date")
dfFootTrafficWeather.head()

In [ ]:
# Merge the Max Temp
dfMaxTempToMerge = max_temp[["max_temp", "Quality", "date"]]
dfMaxTempToMerge.rename(columns={"Quality":"max_temp_quality"},inplace=True)
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfMaxTempToMerge, how="inner", on="date")

# Merge the Max Temp
dfMinTempToMerge = min_temp[["min_temp", "Quality", "date"]]
dfMinTempToMerge.rename(columns={"Quality":"min_temp_quality"},inplace=True)
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfMinTempToMerge, how="inner", on="date")

dfFootTrafficWeather.head()

In [ ]:
solar_exp.head()

In [ ]:
# Merge the Solar Exposure
dfSolarExpToMerge = solar_exp[["solar_exp", "date"]]
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfSolarExpToMerge, how="inner", on="date")

print(dfFootTrafficWeather.info())
dfFootTrafficWeather.head()

Add the day of the week as a feature

In [ ]:
dfFootTrafficWeather["WeekDay"] = dfFootTrafficWeather.apply(lambda x: x["date"].weekday(), axis=1)

Adding annual population and growth rating to the data

In [ ]:
# First, create a temp dateyear column for joining
dfFootTrafficWeather["date_year"] = dfFootTrafficWeather.apply(lambda x: x["date"].year, axis=1)

In [ ]:
dfFootTrafficWeather.head()

In [ ]:
# Load the population file
dfPop = pd.read_csv("./data_files/greatermelb_population_annual.csv")
print(dfPop.shape)
print(dfPop.info())
dfPop.head()

In [ ]:
# Convert to just a date year column
dfPop["date_year"] = dfPop.apply(lambda x: pd.to_datetime(x["date"]).year, axis=1)
dfPop = dfPop.rename(columns={ " Population" : "population_annual", " Annual Change" : "population_change_annual" })
del dfPop["date"]
dfPop.head()

In [ ]:
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfPop, on="date_year")
del dfFootTrafficWeather["date_year"]


In [ ]:
# Load and join the Holidays file
dfHol = pd.read_csv("./data_files/Holidays_20130101_20220701.csv")
dfHol["date"] = pd.to_datetime(dfHol["date"])
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfHol, on="date", how="left")

In [ ]:
# Load and join the Lockdown file
dfLockdown = pd.read_csv("./data_files/Melb_Lockdown_Dates.csv")
dfLockdown["date"] = pd.to_datetime(dfLockdown["date"])
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfLockdown, on="date", how="left")

Create a Date Year-Month key for joining monthly data files like retain and all ords

In [ ]:
dfFootTrafficWeather["date_ym"] = dfFootTrafficWeather.apply(lambda x: str(pd.to_datetime(x["date"]).year) + "-" + str(pd.to_datetime(x["date"]).month), axis=1)

In [ ]:
# Read in the Retail data and join it to our foot traffic data
dfRetail = pd.read_csv("./data_files/OFFLINE_Retail_Turnover_VIC.csv")

dfRetail["date_ym"] = dfRetail.apply(lambda x: str(pd.to_datetime(x["date"]).year) + "-" + str(pd.to_datetime(x["date"]).month), axis=1)
dfRetail=dfRetail.rename(columns= {"Original_Turnover":"OfflineRetail_Original_Turnover", "Seasonally_Adjusted_Turnover":"OfflineRetail_Seasonally_Adjusted_Turnover","Trend_Turnover":"OfflineRetail_Trend_Turnover"})
del dfRetail["date"]
dfRetail.head()

In [ ]:
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfRetail, on="date_ym")

In [ ]:
# Read in the All Ords data and join it to our foot traffic data
dfAllOrds = pd.read_csv("./data_files_raw/aus-all-ords.csv")

#dfAllOrds["date_ym"] = dfAllOrds["Year"] + "-" + dfAllOrds["Month Num"]
dfAllOrds["date_ym"] = dfAllOrds.apply(lambda x: str(x["Year"]) + "-" + str(x["Month Num"]), axis=1)
del dfAllOrds["Year"]
del dfAllOrds["Month Name"]
del dfAllOrds["Month Num"]
dfAllOrds.head()

In [ ]:
dfFootTrafficWeather = pd.merge(dfFootTrafficWeather, dfAllOrds, on="date_ym")

In [ ]:
del dfFootTrafficWeather["date_ym"] 
print(dfFootTrafficWeather.info())
dfFootTrafficWeather.head(20)

Detect any duplicate rows and remove

In [ ]:
print(dfFootTrafficWeather.shape)
dfFootTrafficWeather.groupby('date').filter(lambda group: len(group) > 1).groupby('date').size().head(20)

In [ ]:
dfFootTrafficWeather = dfFootTrafficWeather.drop_duplicates()
print(dfFootTrafficWeather.shape)
dfFootTrafficWeather.groupby('date').filter(lambda group: len(group) > 1).groupby('date').size().head()

Write out all the joined data to file

In [ ]:
outputFootTrafficWeatherFileName = "FT_Street_Melb_" + filesStartDate.strftime("%Y%m%d") + "_" + filesEndDate.strftime("%Y%m%d") + ".csv"
dfFootTrafficWeather.to_csv("./data_files/" + outputFootTrafficWeatherFileName, index=False)